The particle dynamics in a general velocity field $ \mathbf{u}(\mathbf{x}, t) = \begin{pmatrix} u(\mathbf{x}, t) \\ v(\mathbf{x}, t) \end{pmatrix} $. This function evaluates the velocity field $ \mathbf{u}(\mathbf{x}, t) $, at point $ \mathbf{x} $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| Interpolant_u | object | Interpolant object for $ u(\mathbf{x}, t) $ |
| Interpolant_v | object | Interpolant object for $ v(\mathbf{x}, t) $ |
| time_data | array(1,Nt) | time of velocity data |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| vel | array (2,) | $ \mathbf{v}(\mathbf{x}, t) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

In [3]:
def velocity(t, r, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data):
    
    # check if periodic in x
    if periodic[0]:
        
        r[0,:] = (r[0,:]-X[0,0])%(X[0, -1]-X[0, 0])+X[0,0]
    
    # check if periodic in y
    if periodic[1]:
        
        r[1,:] = (r[1,:]-Y[0,0])%(Y[-1, 0]-Y[0, 0])+Y[0,0]
        
    if periodic[2]:
        
        t = t%(time_data[0,-1]-time_data[0,0])+time_data[0,0]
    
    dt_data = time_data[0,1]-time_data[0,0]
    
    # Unsteady case
    if bool_unsteady:

        k = int(t/dt_data)
    
        # evaluate velocity field at time t
        if k >= len(Interpolant_u)-1:
            u = Interpolant_u[-1](r[1,:], r[0,:], grid = False)
            v = Interpolant_v[-1](r[1,:], r[0,:], grid = False)
            
        else: 
    
            ui = Interpolant_u[k](r[1,:], r[0,:], grid = False)
            uf = Interpolant_u[k+1](r[1,:], r[0,:], grid = False)
            u = ((k+1)*dt_data-t)/dt_data*ui + (t-k*dt_data)/dt_data*uf

            vi = Interpolant_v[k](r[1,:], r[0,:], grid = False)
            vf = Interpolant_v[k+1](r[1,:], r[0,:], grid = False)
            v = ((k+1)*dt_data-t)/dt_data*vi + (t-k*dt_data)/dt_data*vf
        
    # Steady case        
    elif bool_unsteady == False:
            
        u = Interpolant_u(r[:,1], r[:,0], grid = False)
        v = Interpolant_v(r[:,1], r[:,0], grid = False)
    
    return np.array([u, v])